In [1]:
from seeq import spy
import pandas as pd
import numpy as np

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../../credentials.key', force=False)

In [3]:
from seeq.spy.assets import Asset

class Area(Asset):
    def initialize(self):
        # These are effectively x,y coordinates proportional to the
        # map of the U.S. in this folder.
        map_positions = {
            # Cooling Tower 1
            'A': ['15%', '4%'],
            'B': ['45%', '60%'],
            'C': ['35%', '35%'],
            'G': ['70%', '60%'],
            'H': ['32%', '80%'],
            'I': ['55%', '40%'],
            'J': ['15%', '34%'],
            'K': ['80%', '6%'],
            
            # Cooling Tower 2
            'D': ['40%', '50%'],
            'E': ['50%', '60%'],
            'F': ['60%', '70%']
        }
        
        self.map_position = map_positions[self['Name']]
    
    @Asset.Attribute()
    def Incidents(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': '$a.validValues().valueSearch(isGreaterThan(120))',
            'Formula Parameters': {
                '$a': metadata[metadata['Name'] == 'Temperature']
            }
        }
    
    @Asset.Attribute()
    def Incident_Count(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Incidents(),
            'Measured Item Maximum Duration': '40d',
            'Statistic': 'count',
            'Thresholds': {
                'HiHiHi': 1,
                'HiHi': 0
            }
        }
    
    @Asset.Display()
    def Incident_Display(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = [{
            'Item': self.Incident_Count()
        }]
        workstep.view = 'Scorecard'
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardHeaders'] = {
          "type": "none"
        }
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardColumns'] = list()
        
        return workstep
    
    @Asset.Display()
    def Detail(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = metadata[metadata['Name'] == 'Temperature']
        workstep.view = 'Trend'
        
        return workstep

class Cooling_Tower(Asset):
    @Asset.Component()
    def Areas(self, metadata):
        return self.build_components(Area, metadata, column_name='Area')
    
    @Asset.DateRange()
    def Last_Two_Months(self, metadata):
        return {
            'Start': '2019-11-01',
            'End': '2020-01-01'
        }
    
    @Asset.DateRange()
    def Last_Week(self, metadata):
        return {
            'Start': '2020-01-01',
            'End': '2020-01-08'
        }
        
    @Asset.Document()
    def Topic_Document(self, metadata, topic):
        document = topic.document('Cooling Tower ' + self.definition['Name'])
        document.render_template(filename='Cooling_Tower_Health.html', asset=self)
        
class Region(Asset):    
    @Asset.Component()
    def Cooling_Towers(self, metadata):
        return self.build_components(Cooling_Tower, metadata, column_name='Cooling Tower')

In [4]:
search_df = spy.search({
    'Path': 'Example >> Cooling Tower ? >> /Area [A-K]/',
    'Type': 'Signal',
    'Name': 'Temperature'})

search_df

,Path,Type,Name,Time,Count,Result
0,Example,Signal,Temperature,00:00:00.03,10,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,431DDA45-599F-48A0-B372-B7F6D3CB7950,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False
1,E7D596B8-EBEA-499D-9F34-7B757924DA77,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False
2,AACE5B6B-072D-4871-B23D-9DF5A34949C2,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False
3,53084E49-A1B4-41D1-AFCA-B5F59050E86F,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False
4,2BC6CBB0-D5D8-487F-BF24-FAC90B3A06B6,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False
5,EFF0535A-D724-41AF-AB1E-44180E7B28C1,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False
6,EA9C0B72-48B4-4C9D-9D5B-5622D180AF26,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False
7,B2FE2617-1603-4C84-99F3-AF6C4FC867FB,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False
8,EA021BE4-D11E-4505-8E85-8029F24793D9,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False
9,ED418D42-2E2A-46D9-A1DD-D2815234D915,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False


In [5]:
metadata_df = search_df.copy()
metadata_df['Cooling Tower'] = metadata_df['Path'].str.extract(r'Cooling Tower (\d)')
metadata_df['Area'] = metadata_df['Asset'].str.extract(r'Area (\w)')
metadata_df['Build Path'] = 'Cooling Tower Health'
metadata_df['Build Asset'] = 'North America'
metadata_df

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Cooling Tower,Area,Build Path,Build Asset
0,431DDA45-599F-48A0-B372-B7F6D3CB7950,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False,2,D,Cooling Tower Health,North America
1,E7D596B8-EBEA-499D-9F34-7B757924DA77,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False,1,I,Cooling Tower Health,North America
2,AACE5B6B-072D-4871-B23D-9DF5A34949C2,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False,1,C,Cooling Tower Health,North America
3,53084E49-A1B4-41D1-AFCA-B5F59050E86F,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False,1,G,Cooling Tower Health,North America
4,2BC6CBB0-D5D8-487F-BF24-FAC90B3A06B6,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False,1,K,Cooling Tower Health,North America
5,EFF0535A-D724-41AF-AB1E-44180E7B28C1,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False,1,J,Cooling Tower Health,North America
6,EA9C0B72-48B4-4C9D-9D5B-5622D180AF26,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False,1,A,Cooling Tower Health,North America
7,B2FE2617-1603-4C84-99F3-AF6C4FC867FB,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False,2,E,Cooling Tower Health,North America
8,EA021BE4-D11E-4505-8E85-8029F24793D9,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False,1,H,Cooling Tower Health,North America
9,ED418D42-2E2A-46D9-A1DD-D2815234D915,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False,1,B,Cooling Tower Health,North America


In [6]:
build_df = spy.assets.build(Region, metadata_df)
build_df

,Type,Formula,Formula Parameters,Name,Asset,Asset Object,Path,Template,Build Result,Measured Item,Measured Item Maximum Duration,Statistic,Thresholds,Workbook Type,Object
0,Condition,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},Incidents,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN
1,Metric,NaN,NaN,Incident Count,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN
2,Asset,NaN,NaN,D,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN
3,Condition,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},Incidents,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN
4,Metric,NaN,NaN,Incident Count,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN
5,Asset,NaN,NaN,E,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN
6,Asset,NaN,NaN,2,2,Cooling Tower Health >> North America >> 2,Cooling Tower Health >> North America,Cooling Tower,Success,NaN,NaN,NaN,NaN,NaN,NaN
7,Condition,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},Incidents,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN
8,Metric,NaN,NaN,Incident Count,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN
9,Asset,NaN,NaN,I,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
spy.push(metadata=build_df)

,Type,Formula,Formula Parameters,Name,Asset,Asset Object,Path,Template,Build Result,Measured Item,Measured Item Maximum Duration,Statistic,Thresholds,Workbook Type,Object,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,CalculatedCondition,$a.validValues().valueSearch(isGreaterThan(120)),[a=431DDA45-599F-48A0-B372-B7F6D3CB7950],Incidents,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Condit...,A1668C45-5E76-423E-B584-1C5436F70D4F,Success
1,ThresholdMetric,NaN,NaN,Incident Count,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Metric...,E777E96D-21A2-4EE3-AC0A-EF051A3FF572,Success
2,Asset,NaN,NaN,D,D,Cooling Tower Health >> North America >> 2 >> D,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Asset}...,0C430E3B-6D26-435D-90DF-F05F3FD131DF,Success
3,CalculatedCondition,$a.validValues().valueSearch(isGreaterThan(120)),[a=B2FE2617-1603-4C84-99F3-AF6C4FC867FB],Incidents,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Condit...,11FA9084-3637-4DEE-B111-F510575C7928,Success
4,ThresholdMetric,NaN,NaN,Incident Count,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Metric...,8E909687-348D-4C61-B3CE-223CBC549376,Success
5,Asset,NaN,NaN,E,E,Cooling Tower Health >> North America >> 2 >> E,Cooling Tower Health >> North America >> 2,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Asset}...,5FF7A3C7-09A0-4324-A7E7-2443F81FF83C,Success
6,Asset,NaN,NaN,2,2,Cooling Tower Health >> North America >> 2,Cooling Tower Health >> North America,Cooling Tower,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Asset}...,04E6438C-989C-436F-A7D7-86491737C925,Success
7,CalculatedCondition,$a.validValues().valueSearch(isGreaterThan(120)),[a=E7D596B8-EBEA-499D-9F34-7B757924DA77],Incidents,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Condit...,F1C36410-3197-4F2E-A3A6-3D5FBD47C8F6,Success
8,ThresholdMetric,NaN,NaN,Incident Count,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,count,"{'HiHiHi': 1, 'HiHi': 0}",NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Metric...,47C1E3C1-A7FA-41E2-9915-6BC27E6D77FE,Success
9,Asset,NaN,NaN,I,I,Cooling Tower Health >> North America >> 1 >> I,Cooling Tower Health >> North America >> 1,Area,Success,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[E965F8DF-EC82-4BF7-AE43-22E6ADA5B20B] {Asset}...,6AE1DFD5-0191-48AD-85D0-AF3049256398,Success
